In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [2]:

dataset=pd.read_csv('../dataset_unified.csv', sep=';')
dataset.head()

,company,date,quarter,country,ROA,ROE,debt_to_equity,current_ratio,net_margin,revenue_growth,cash_ratio,inflation_YoY,gdp_growth_rate,interest_rate
0,Banco Santander,2024-09-30,2024-Q3,Spain,NaN,NaN,"16,15","0,33",NaN,"6,2414",NaN,"0,0217","0,0080","0,0365"
1,Banco Santander,2024-06-30,2024-Q2,Spain,"0,63","0,109","16,23","0,23","0,0906","-0,8712","0,23","0,0346","0,0080","0,0425"
2,Banco Santander,2024-03-31,2024-Q1,Spain,"0,6","0,1043","16,14","0,24","0,087","-0,6185","0,24","0,0314","0,0100","0,0450"
3,Banco Santander,2023-12-31,2023-Q4,Spain,"0,59","0,1035","16,24","0,35","0,0886","22,1927","0,34","0,0327","0,0070","0,0450"
4,Banco Santander,2023-09-30,2023-Q3,Spain,"0,55","0,0968","16,66","0,31","0,1044","0,0201","0,31","0,0282","0,0070","0,0450"


In [3]:
indicators_to_clean = [
    'ROA', 'ROE', 'net_margin', 'current_ratio',
    'cash_ratio', 'debt_to_equity', 'revenue_growth',
    'inflation_YoY', 'gdp_growth_rate', 'interest_rate'
]

# STEP 1: Clean all comma-based numbers → dots, then convert to numeric
for col in indicators_to_clean:
    dataset[col] = (
        dataset[col]
        .astype(str)
        .str.replace(',', '.', regex=False)     # Convert commas to dots
        .replace('nan', np.nan)                 # Replace string 'nan' with real NaN
    )
    dataset[col] = pd.to_numeric(dataset[col], errors='coerce')  # Convert to float

# ✅ Optional: check if conversion worked
print(dataset[indicators_to_clean].dtypes)

ROA                float64
ROE                float64
net_margin         float64
current_ratio      float64
cash_ratio         float64
debt_to_equity     float64
revenue_growth     float64
inflation_YoY      float64
gdp_growth_rate    float64
interest_rate      float64
dtype: object


In [4]:
dataset['ROA']=dataset['ROA']/100
dataset.head()

,company,date,quarter,country,ROA,ROE,debt_to_equity,current_ratio,net_margin,revenue_growth,cash_ratio,inflation_YoY,gdp_growth_rate,interest_rate
0,Banco Santander,2024-09-30,2024-Q3,Spain,NaN,NaN,16.15,0.33,NaN,6.2414,NaN,0.0217,0.008,0.0365
1,Banco Santander,2024-06-30,2024-Q2,Spain,0.0063,0.1090,16.23,0.23,0.0906,-0.8712,0.23,0.0346,0.008,0.0425
2,Banco Santander,2024-03-31,2024-Q1,Spain,0.0060,0.1043,16.14,0.24,0.0870,-0.6185,0.24,0.0314,0.010,0.0450
3,Banco Santander,2023-12-31,2023-Q4,Spain,0.0059,0.1035,16.24,0.35,0.0886,22.1927,0.34,0.0327,0.007,0.0450
4,Banco Santander,2023-09-30,2023-Q3,Spain,0.0055,0.0968,16.66,0.31,0.1044,0.0201,0.31,0.0282,0.007,0.0450


In [5]:
dataset[dataset['company']=='BNP Paribas']['current_ratio'].describe()

count    43.000000
mean      0.779070
std       0.072893
min       0.680000
25%       0.735000
50%       0.760000
75%       0.810000
max       1.040000
Name: current_ratio, dtype: float64

In [6]:
import pandas as pd

# Step 1 — Clip only for Banco Santander
dataset['current_ratio_clipped'] = dataset['current_ratio']  # Start with original values

# Apply clipping only for Banco Santander
santander_mask = dataset['company'] == 'Banco Santander'
dataset.loc[santander_mask, 'current_ratio_clipped'] = dataset.loc[santander_mask, 'current_ratio'].clip(lower=0, upper=2.74)

# Step 2 — Normalization function per company
def normalize_local(df, columns):
    df_copy = df.copy()
    for col in columns:
        df_copy[col + '_localc'] = df_copy.groupby('company')[col].transform(
            lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0.5
        )
    return df_copy

# Step 3 — List of indicators to normalize
indicators_to_normalize = [
    'ROA', 'ROE', 'net_margin',
    'current_ratio_clipped', 'cash_ratio',
    'debt_to_equity'
]

# Step 4 — Normalize
dataset_normalized = normalize_local(dataset, indicators_to_normalize)

# Step 5 — Rename for consistency
dataset_normalized.rename(columns={'current_ratio_clipped_localc': 'current_ratio_localc'}, inplace=True)

# Step 6 — View the result
dataset_normalized.head()


,company,date,quarter,country,ROA,ROE,debt_to_equity,current_ratio,net_margin,revenue_growth,...,inflation_YoY,gdp_growth_rate,interest_rate,current_ratio_clipped,ROA_localc,ROE_localc,net_margin_localc,current_ratio_localc,cash_ratio_localc,debt_to_equity_localc
0,Banco Santander,2024-09-30,2024-Q3,Spain,NaN,NaN,16.15,0.33,NaN,6.2414,...,0.0217,0.008,0.0365,0.33,NaN,NaN,NaN,0.047431,NaN,0.790353
1,Banco Santander,2024-06-30,2024-Q2,Spain,0.0063,0.1090,16.23,0.23,0.0906,-0.8712,...,0.0346,0.008,0.0425,0.23,1.000000,1.000000,0.268603,0.007905,0.007905,0.805195
2,Banco Santander,2024-03-31,2024-Q1,Spain,0.0060,0.1043,16.14,0.24,0.0870,-0.6185,...,0.0314,0.010,0.0450,0.24,0.938776,0.946163,0.250255,0.011858,0.011858,0.788497
3,Banco Santander,2023-12-31,2023-Q4,Spain,0.0059,0.1035,16.24,0.35,0.0886,22.1927,...,0.0327,0.007,0.0450,0.35,0.918367,0.936999,0.258410,0.055336,0.051383,0.807050
4,Banco Santander,2023-09-30,2023-Q3,Spain,0.0055,0.0968,16.66,0.31,0.1044,0.0201,...,0.0282,0.007,0.0450,0.31,0.836735,0.860252,0.338940,0.039526,0.039526,0.884972


In [7]:
dataset_normalized[dataset_normalized['company']=='BNP Paribas']['current_ratio_localc'].describe()

count    43.000000
mean      0.275194
std       0.202481
min       0.000000
25%       0.152778
50%       0.222222
75%       0.361111
max       1.000000
Name: current_ratio_localc, dtype: float64

In [8]:
#Score 1 : profitability
dataset_normalized['score_profitability'] = dataset_normalized[['ROA_localc', 'ROE_localc', 'net_margin_localc']].mean(axis=1)

# Score 2 : Liquidity
dataset_normalized['score_liquidity'] = dataset_normalized[['current_ratio_localc', 'cash_ratio_localc']].mean(axis=1)    

# Score 3 : solvency
dataset_normalized['score_solvency']=1-dataset_normalized['debt_to_equity_localc']

# Score 4 :  Leverage-Adjusted Profitability
dataset_normalized['inv_debt_to_equity_localc'] = 1 - dataset_normalized['debt_to_equity_localc']
dataset_normalized['score_leverage_ajusted']=dataset_normalized[['ROE_localc','inv_debt_to_equity_localc']].mean(axis=1)

print(dataset_normalized[dataset_normalized['company'] == 'Crédit Agricole']['score_leverage_ajusted'].to_list())


[0.874707536933907, 0.9368331199317116, 0.9342723004694836, 0.9487836107554417, 0.8604073142763113, 0.8950064020486556, 0.8783610755441742, 0.8868971404182672, 0.7855428114506682, 0.871105420401195, 0.9065300896286812, 0.918907383696116, 0.8686642858244757, 0.8352539479300043, 0.8169014084507042, 0.7733674775928296, 0.7921641752670091, 1.0, 0.9526248399487837, 0.9393939393939394, 0.960734101579172, 0.9316439895304649, 0.9250907599668841, 0.9489970123772941, 0.8614493826779589, 0.7942808365343577, 0.8121483372671222, 0.5413090261172732, 0.09463873419413874, 0.028092266633756403, 0.0, 0.0894804570394252, 0.44019895201805925]


In [9]:
dataset_normalized[dataset_normalized['company']=='HSBC']['score_liquidity'].describe()

count    37.000000
mean      0.545270
std       0.225002
min       0.062500
25%       0.412500
50%       0.537500
75%       0.700000
max       0.968750
Name: score_liquidity, dtype: float64

In [10]:


# Sample structure — replace this with your real normalized data
df = dataset_normalized

# Step 1: Score threshold classifier
def classify_score(score):
    if pd.isna(score):
        return "Missing"
    elif score < 0.2:
        return "Red"
    elif score < 0.7:
        return "Yellow"
    else:
        return "Green"

# Step 2: Revenue growth signal
def classify_revenue_alert(value):
    if pd.isna(value):
        return "Unknown"
    elif value < -10:
        return "Rev ↓"
    elif value > 10:
        return "Rev ↑"
    else:
        return "Stable"

# Step 3: Combine into single alert summary
def build_alert(row):
    alerts = []
    if classify_score(row["score_profitability"]) == "Red":
        alerts.append("Profit")
    if classify_score(row["score_liquidity"]) == "Red":
        alerts.append("Liquidity")
    if classify_score(row["score_solvency"]) == "Red":
        alerts.append("Solvency")
    if classify_score(row["score_leverage_ajusted_localc"]) == "Red":
        alerts.append("Adj. Leverage")

    alert_text = f"Red ({', '.join(alerts)})" if alerts else ""

    rev_flag = classify_revenue_alert(row["revenue_growth"])
    if rev_flag in ["Rev ↓", "Rev ↑"]:
        alert_text += f", {rev_flag}" if alert_text else rev_flag

    return alert_text if alert_text else "Stable"

# Apply to DataFrame
df["Alert Summary"] = df.apply(build_alert, axis=1)

# Step 4: Final table format
final_table = df[[
    "quarter",
    "score_profitability",
    "score_liquidity",
    "score_solvency",
    "revenue_growth",
    "Alert Summary"
]].rename(columns={
    "quarter": "Quarter",
    "score_profitability": "Profitability",
    "score_liquidity": "Liquidity",
    "score_solvency": "Solvency",
    "revenue_growth": "Rev Growth"
})

# Optional: Format revenue growth as %
final_table["Rev Growth"] = final_table["Rev Growth"].apply(lambda x: f"{x:.1f}%" if pd.notna(x) else "NaN")

# Show table
print(final_table.to_string(index=False))


KeyError: 'score_leverage_ajusted_localc'

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output


company_dropdown = widgets.Dropdown(
    options=sorted(dataset_normalized['company'].unique()),
    description='Company:',
    value='Banco Santander'  
)

output = widgets.Output()

def classify_revenue_alert(value):
    if pd.isna(value):
        return "Unknown"
    elif value < -0.10:
        return "Rev ↓"
    elif value > 0.10:
        return "Rev ↑"
    else:
        return "Stable"

def alert_summary(row):
    indicators = {
        'Profit': row.get('score_profitability'),
        'Liquidity': row.get('score_liquidity'),
        'Solvency': row.get('score_solvency'),
        'Adj. Leverage': row.get('score_leverage_ajusted')
    }

    reds = [k for k, v in indicators.items() if pd.notna(v) and v < 0.2]
    greens = [k for k, v in indicators.items() if pd.notna(v) and v > 0.7]
    stables = [k for k, v in indicators.items() if pd.notna(v) and 0.2 <= v <= 0.7]

    alert_parts = []
    if reds:
        alert_parts.append(f"Red ({', '.join(reds)})")
    if greens:
        alert_parts.append(f"Green ({', '.join(greens)})")
    if not reds and not greens and stables:
        alert_parts.append(f"Stable ({', '.join(stables)})")
    if not alert_parts:
        alert_parts.append("Missing")

    rev = classify_revenue_alert(row.get('revenue_growth'))
    if rev in ["Rev ↓", "Rev ↑"]:
        alert_parts.append(rev)

    return ", ".join(alert_parts)
    return alert




def global_status(row):
    indicators = [
        row.get('score_profitability'),
        row.get('score_liquidity'),
        row.get('score_solvency'),
        row.get('score_leverage_ajusted')
    ]
    valid_scores = [s for s in indicators if pd.notna(s)]
    rev = classify_revenue_alert(row.get('revenue_growth'))

    red = sum(1 for s in valid_scores if s < 0.2)
    green = sum(1 for s in valid_scores if s > 0.7)

    if red >= 2:
        return "Danger"
    elif green >= 2 and red == 0:
        return "Strong" if rev != "Rev ↑" else "Excellent"
    elif green == 1 and red == 1:
        return "Mixed Risk"
    elif red == 1 and green == 0:
        return "Caution"
    elif len(valid_scores) == 4 and all(0.2 <= s <= 0.7 for s in valid_scores):
        return "Stable"
    else:
        return "Positive signal"



def update_table(change):
    with output:
        clear_output()
        selected_company = change['new']
        filtered = dataset_normalized[dataset_normalized['company'] == selected_company].copy()
        filtered["Rev Growth"] = filtered["revenue_growth"].apply(lambda x: f"{x:.1f}%" if pd.notna(x) else "")
        filtered["Alert Summary"] = filtered.apply(alert_summary, axis=1)
        filtered["Overall Status"] = filtered.apply(global_status, axis=1)

        summary = filtered[[ 
            "quarter", "score_profitability", "score_liquidity", "score_solvency","score_leverage_ajusted", "Rev Growth", "Alert Summary", "Overall Status"
        ]].rename(columns={
            "quarter": "Quarter",
            "score_profitability": "Profitability",
            "score_liquidity": "Liquidity",
            "score_solvency": "Solvency",
            "score_leverage_ajusted": "Adj. Leverage",
        }).reset_index(drop=True)
        display(summary)

# Connect widget to update function
company_dropdown.observe(update_table, names='value')

# Display widgets and initial table
display(company_dropdown, output)
update_table({'new': company_dropdown.value})


Dropdown(description='Company:', index=1, options=('BNP Paribas', 'Banco Santander', 'Crédit Agricole', 'HSBC'…

Output()

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

company_dropdown = widgets.Dropdown(
    options=sorted(dataset_normalized['company'].unique()),
    description='Company:',
    value='Banco Santander'  
)

output = widgets.Output()

def classify_revenue_alert(value):
    if pd.isna(value):
        return "Unknown"
    elif value < -0.10:
        return "Rev ↓"
    elif value > 0.10:
        return "Rev ↑"
    else:
        return "Stable"

def alert_summary(row):
    indicators = {
        'Profit': row.get('score_profitability'),
        'Liquidity': row.get('score_liquidity'),
        'Solvency': row.get('score_solvency'),
        'Adj. Leverage': row.get('score_leverage_ajusted')
    }

    reds = [k for k, v in indicators.items() if pd.notna(v) and v < 0.2]
    greens = [k for k, v in indicators.items() if pd.notna(v) and v > 0.7]
    stables = [k for k, v in indicators.items() if pd.notna(v) and 0.2 <= v <= 0.7]

    alert_parts = []
    if reds:
        alert_parts.append(f"Red ({', '.join(reds)})")
    if greens:
        alert_parts.append(f"Green ({', '.join(greens)})")
    if not reds and not greens and stables:
        alert_parts.append(f"Stable ({', '.join(stables)})")
    if not alert_parts:
        alert_parts.append("Missing")

    rev = classify_revenue_alert(row.get('revenue_growth'))
    if rev in ["Rev ↓", "Rev ↑"]:
        alert_parts.append(rev)

    return ", ".join(alert_parts)

def global_status(row):
    scores = {
        'Profit': row.get('score_profitability'),
        'Liquidity': row.get('score_liquidity'),
        'Solvency': row.get('score_solvency'),
        'Adj. Leverage': row.get('score_leverage_ajusted')
    }

    valid_scores = [v for v in scores.values() if pd.notna(v)]
    rev = classify_revenue_alert(row.get('revenue_growth'))
    adj_leverage = scores.get('Adj. Leverage')

    red = sum(1 for s in valid_scores if s < 0.2)
    green = sum(1 for s in valid_scores if s > 0.7)

    if adj_leverage is not None and adj_leverage < 0.2:
        return "Highly Leveraged Risk"
    elif adj_leverage is not None and adj_leverage > 0.7 and red == 0 and rev == "Rev ↑":
        return "Excellent Health"
    elif adj_leverage is not None and adj_leverage > 0.7 and red == 0:
        return "Strong Capital Efficiency"
    elif red >= 3:
        return "Critical Risk"  
    elif red >= 2:
        return "Danger"
    elif green >= 2 and red == 0 and (adj_leverage is None or adj_leverage <= 0.7):
        return "Strong"
    
    elif green >=2 and red <=1 or green >= 1 and red == 0: 
        return "Good signal"

    elif green == red and green != 0:
        return "Mixed Risk"
    elif red == 1 and green == 0:
        return "Caution"
    elif len(valid_scores) == 4 and all(0.2 <= s <= 0.7 for s in valid_scores):
        return "Stable"
    else:
        return "Watch"


def color_overall_status(val):
    color_map = {
        "Excellent Health": 'background-color: #2e7d32; color: white',             # dark green
        "Strong Capital Efficiency": 'background-color: #13c01c; color: white',     # lighter green
        "Strong": 'background-color: #37d310; color: black',                         # medium green
        "Stable": 'background-color: #b7b9bc; color: white',                         # blue
        "Mixed Risk": 'background-color: #FFEB3B; color: black',                     # orange
        "Caution": 'background-color:#FF9800 ; color: black',                        # yellow
        "Danger": 'background-color: #f44336; color: white',                         # red
        "Severe Risk": 'background-color: #b71c1c; color: white',                    # deep red (formerly "Big Danger")
        "Highly Leveraged Risk": 'background-color: #d50000; color: white',          # red accent
        "Good signal": 'background-color: #a5f990; color: black', # light blue
        "Positive signal": 'background-color: #c5e1a5; color: black',                # pale green
        "Watch": 'background-color: #e0e0e0; color: black'                           # grey
    }
    return color_map.get(val, '')

   

def update_table(change):
    with output:
        clear_output()
        selected_company = change['new']
        filtered = dataset_normalized[dataset_normalized['company'] == selected_company].copy()

        filtered["Rev Growth"] = filtered["revenue_growth"].apply(lambda x: f"{x:.1f}%" if pd.notna(x) else "")
        filtered["Alert Summary"] = filtered.apply(alert_summary, axis=1)
        filtered["Overall Status"] = filtered.apply(global_status, axis=1)

        summary = filtered[[ 
            "quarter", "score_profitability", "score_liquidity", "score_solvency", "score_leverage_ajusted",
            "Rev Growth", "Alert Summary", "Overall Status"
        ]].rename(columns={
            "quarter": "Quarter",
            "score_profitability": "Profitability",
            "score_liquidity": "Liquidity",
            "score_solvency": "Solvency",
            "score_leverage_ajusted": "Adj. Leverage",
        }).reset_index(drop=True)

        styled = summary.style.applymap(color_overall_status, subset=["Overall Status"])
        display(styled)

# Connect and display
company_dropdown.observe(update_table, names='value')
display(company_dropdown, output)
update_table({'new': company_dropdown.value})


Dropdown(description='Company:', index=1, options=('BNP Paribas', 'Banco Santander', 'Crédit Agricole', 'HSBC'…

Output()

In [11]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Compute dynamic thresholds for each score
thresholds = {
    'score_profitability': {
        'low': dataset_normalized['score_profitability'].quantile(0.2),
        'high': dataset_normalized['score_profitability'].quantile(0.8)
    },
    'score_liquidity': {
        'low': dataset_normalized['score_liquidity'].quantile(0.2),
        'high': dataset_normalized['score_liquidity'].quantile(0.8)
    },
    'score_solvency': {
        'low': dataset_normalized['score_solvency'].quantile(0.2),
        'high': dataset_normalized['score_solvency'].quantile(0.8)
    },
    'score_leverage_ajusted': {
        'low': dataset_normalized['score_leverage_ajusted'].quantile(0.15),
        'high': dataset_normalized['score_leverage_ajusted'].quantile(0.85)
    }
}

company_dropdown = widgets.Dropdown(
    options=sorted(dataset_normalized['company'].unique()),
    description='Company:',
    value='Banco Santander'
)

output = widgets.Output()

def classify_revenue_alert(value):
    if pd.isna(value):
        return "Unknown"
    elif value < -0.10:
        return "Rev ↓"
    elif value > 0.10:
        return "Rev ↑"
    else:
        return "Stable"

def alert_summary(row):
    reds, greens, stables = [], [], []
    for key, label in [
        ('score_profitability', 'Profit'),
        ('score_liquidity', 'Liquidity'),
        ('score_solvency', 'Solvency'),
        ('score_leverage_ajusted', 'Adj. Leverage')
    ]:
        score = row.get(key)
        if pd.isna(score):
            continue
        if score < thresholds[key]['low']:
            reds.append(label)
        elif score > thresholds[key]['high']:
            greens.append(label)
        else:
            stables.append(label)

    alert_parts = []
    if reds:
        alert_parts.append(f"Red ({', '.join(reds)})")
    if greens:
        alert_parts.append(f"Green ({', '.join(greens)})")
    if not reds and not greens and stables:
        alert_parts.append(f"Stable ({', '.join(stables)})")
    if not alert_parts:
        alert_parts.append("Missing")

    rev = classify_revenue_alert(row.get('revenue_growth'))
    if rev in ["Rev ↓", "Rev ↑"]:
        alert_parts.append(rev)

    return ", ".join(alert_parts)

def global_status(row):
    indicators = {
        'score_profitability': row.get('score_profitability'),
        'score_liquidity': row.get('score_liquidity'),
        'score_solvency': row.get('score_solvency'),
        'score_leverage_ajusted': row.get('score_leverage_ajusted')
    }

    rev = classify_revenue_alert(row.get('revenue_growth'))
    red = sum(1 for k, v in indicators.items()
              if pd.notna(v) and v < thresholds[k]['low'])
    green = sum(1 for k, v in indicators.items()
                if pd.notna(v) and v > thresholds[k]['high'])
    adj_leverage = indicators['score_leverage_ajusted']

    if adj_leverage is not None and adj_leverage < thresholds['score_leverage_ajusted']['low']:
        return "Leveraged Risk"
    elif adj_leverage is not None and adj_leverage > thresholds['score_leverage_ajusted']['high'] and red == 0 and rev == "Rev ↑":
        return "Excellent Health"
    elif adj_leverage is not None and adj_leverage > thresholds['score_leverage_ajusted']['high'] and red == 0:
        return "Strong Capital Efficiency"
    elif red >= 3:
        return "Critical Risk"
    elif red >= 2:
        return "Danger"
    elif green >= 2 and red == 0 and adj_leverage <= thresholds['score_leverage_ajusted']['high']:
        return "Strong"
    elif green >= 2 and red <= 1 or (green >= 1 and red == 0):
        return "Good signal"
    elif red == green and red > 0:
        return "Mixed Risk"
    elif red == 1 and green == 0:
        return "Caution"
    elif all(0.2 <= v <= 0.7 for v in indicators.values() if pd.notna(v)):
        return "Stable"
    else:
        return "Watch"

def update_table(change):
    with output:
        clear_output()
        selected_company = change['new']
        filtered = dataset_normalized[dataset_normalized['company'] == selected_company].copy()
        filtered["Rev Growth"] = filtered["revenue_growth"].apply(lambda x: f"{x:.1f}%" if pd.notna(x) else "")
        filtered["Alert Summary"] = filtered.apply(alert_summary, axis=1)
        filtered["Overall Status"] = filtered.apply(global_status, axis=1)

        summary = filtered[[
            "quarter", "score_profitability", "score_liquidity", "score_solvency", "score_leverage_ajusted",
            "Rev Growth", "Alert Summary", "Overall Status"
        ]].rename(columns={
            "quarter": "Quarter",
            "score_profitability": "Profitability",
            "score_liquidity": "Liquidity",
            "score_solvency": "Solvency",
            "score_leverage_ajusted": "Adj. Leverage",
        }).reset_index(drop=True)

        display(summary)

# Display
company_dropdown.observe(update_table, names='value')
display(company_dropdown, output)
update_table({'new': company_dropdown.value})


Dropdown(description='Company:', index=1, options=('BNP Paribas', 'Banco Santander', 'Crédit Agricole', 'HSBC'…

Output()